In [1]:
import numpy as np
import cv2
# Functions to generate simple plots


def create_scatterplot_with_coco_bounding_boxes(num_points=10, size=(256, 256), dot_size=3):
    # Initialize an image with ones (white)
    image = np.ones((size[0], size[1], 3), dtype=np.uint8) * 255

    # Generate random points
    points = np.random.randint(0, size[0], size=(num_points, 2))

    # Bounding boxes list
    bounding_boxes = []

    # Drawing points on the image and creating bounding boxes
    for point in points:
        x, y = point
        # Draw a bigger dot
        image[y-dot_size:y+dot_size+1, x-dot_size:x+dot_size+1] = [0, 0, 0]  # Black point

        # Bounding box for each point (in COCO format)
        bb_width = bb_height = 2 * dot_size
        x_center = x / size[0]
        y_center = y / size[1]
        width_normalized = bb_width / size[0]
        height_normalized = bb_height / size[1]

        bounding_box = (x_center, y_center, width_normalized, height_normalized)
        bounding_boxes.append(bounding_box)

    return image, bounding_boxes

def save_coco_bounding_boxes_to_txt(bounding_boxes, filename):
    with open(filename, 'w') as file:
        for box in bounding_boxes:
            x_center, y_center, width, height = box
            x1 = x_center - width / 2
            y1 = y_center - height / 2
            x2 = x_center + width / 2
            y2 = y_center + height / 2
            file.write(f'0 {x1} {y1} {width} {height}\n')

def read_coco_bounding_boxes_from_txt(filename):
    bounding_boxes = []
    with open(filename, 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.strip().split()
            category_id = int(data[0])
            x1, y1, width, height = map(float, data[1:])
            x_center = x1 + width / 2
            y_center = y1 + height / 2
            bounding_box = (x_center, y_center, width, height)
            bounding_boxes.append(bounding_box)
    return bounding_boxes

def visualize_coco_bounding_boxes(image, bounding_boxes, size=(256, 256)):
    # Iterate over the bounding boxes and draw them
    for box in bounding_boxes:
        x_center, y_center, width, height = box
        x_center *= size[0]
        y_center *= size[1]
        width *= size[0]
        height *= size[1]

        # Convert COCO format to top-left and bottom-right coordinates
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Draw rectangle (bounding box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 1)  # Red box

    return image

# Generate image and bounding boxes
scatter_image, coco_boxes = create_scatterplot_with_coco_bounding_boxes()

# # Save the generated image
# image_path = 'scatterplot.png'
# cv2.imwrite(image_path, scatter_image)

# # Save the bounding boxes to a text file in COCO format
# bounding_boxes_path = 'bounding_boxes.txt'
# save_coco_bounding_boxes_to_txt(coco_boxes, bounding_boxes_path)

# # Read the image and bounding box information from the files
# loaded_image = cv2.imread(image_path)
# loaded_coco_boxes = read_coco_bounding_boxes_from_txt(bounding_boxes_path)

# # Visualize the bounding boxes on the loaded image
# visualized_image = visualize_coco_bounding_boxes(loaded_image.copy(), loaded_coco_boxes)

# # Display or save the visualized image
# visualized_image_path = 'visualized_coco_scatterplot.png'
# cv2.imwrite(visualized_image_path, visualized_image)


In [8]:
# Create folders to fill in with data
import os
base_dir='yolo'
dirs =  ['train', 'test', 'val']
for subset in dirs:
    img_dir = os.path.join(base_dir, 'images', subset)
    label_dir = os.path.join(base_dir, 'labels', subset)
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)


In [5]:
from ultralytics import YOLO
# Train the model with Ultralytics - wrap around of Pytorch Yolo model
# yolov8s-p2 - version for detecting smaller objects on images
# https://github.com/ultralytics/ultralytics/issues/981  - you can find out more here
def train_model():
    # Load the pre-trained model
    model = YOLO("yolov8s-p2.yaml").load("yolov8s.pt")

    # Train the model
    model.train(
        data="dataset.yaml", epochs=200,  imgsz=320, save=True, format='onnx'
    )  # Set imgsz to 320 for training on 320xsomething images

    # Export the model to ONNX format
    path = model.export()
    print(path)


In [8]:
# I WOULD HIGHLY RECCOMEND TRAINING USING COMET (its like MLFLOW)
# https://docs.ultralytics.com/modes/train/#logging - HOW TO
# you will have all the weights and training progress save there inside of experiments


In [9]:
train_model()



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                


  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

100%|██████████| 755k/755k [00:00<00:00, 9.79MB/s]


Overriding model.yaml nc=80 with nc=21

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 14.2MB/s]


...

     41/200      2.71G        nan        nan        nan        269        320: 100%|██████████| 7/7 [00:04<00:00,  1.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.84it/s]

                   all         12        336          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/7 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [3]:
# Example of predictions
from ultralytics import YOLO

model = YOLO('best-30.pt') # Load this from
results = model('ObjectRecognition/yolo/dataset/test/0000.jpg', imgsz=256, save=True, conf=0.5)


FileNotFoundError: [Errno 2] No such file or directory: 'best-30.pt'